In [10]:
import pandas as pd

data = pd.read_csv('mintaka_paper_res_sel_rank.csv')

In [11]:
import requests
import time
from tqdm import tqdm

def return_props(entity_idx, entity_idx2):
    url = "https://query.wikidata.org/sparql"
    query = """

      SELECT DISTINCT ?property  WHERE {

        wd:<ENTITY> ?property wd:<ENTITY2> .}
            """.replace(
                "<ENTITY>", entity_idx).replace(
                "<ENTITY2>", entity_idx2)
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    subjects = set()
    for q in range(len(data["results"]['bindings'])):
      if 'http://www.wikidata.org/prop/direct/' in data["results"]['bindings'][q]['property']['value']:
        subjects.add(data["results"]['bindings'][q]['property']['value'].split('http://www.wikidata.org/prop/direct/')[-1])


    subj_list = list(subjects)

    return subj_list

In [51]:
for i in tqdm(range(len(data))):
    if pd.isna(data.loc[i, 'true_props']):
        #and len(data.loc[i, 'ents_lbls'].split(', ')) > 2
        subj = data.loc[i, 'ents_lbls'].split(', ')[2]
        obj = data.loc[i, 'answerLabel']
        time.sleep(1)
        props = return_props(obj, subj)
        if len(props) > 0:
            for k in range(len(props)):
                props[k] = "R"+props[k][1:]
            data.loc[i, 'true_props'] = ', '.join(props)
        else:
            data.loc[i, 'true_props'] = np.nan

100%|██████████| 340/340 [00:00<00:00, 49106.87it/s]


In [53]:
data.head()

,id,question,entities,category,complexityType,answerType,answerLabel,questionType,ents_lbls,ents_len,...,scores_en_rank,ids_one_hops_dict_en,subj_rank,rel_rank,rel_rank_id,ans_rank,ans_rank_clean,pred_obj_maxpred,ans_class_rank,true_props
0,15aae099,Who is George Clooney's current wife?,"[{'name': 'Q23844', 'entityType': 'entity', 'l...",movies,generic,entity,Q16769592,simple,Q23844,1.0,...,"0.8600443601608276, 0.011083224788308144","{'Q23844': 'P463, P5007, P8309, P22, P3341, P5...",Q23844,spouse,P26,http://www.wikidata.org/entity/Q16769592,Q16769592,http://www.wikidata.org/entity/Q16769592,Q16769592,P26
1,da14605c,Who got the horse head in The Godfather?,"[{'name': 'Q47703', 'entityType': 'entity', 'l...",movies,generic,entity,Q1159541,simple,Q47703,1.0,...,"0.8363757729530334, 0.06390717625617981, 0.035...","{'Q47703': 'P11460, P462, P11190, P1651, P1237...",Q47703,director of photography,P344,http://www.wikidata.org/entity/Q560108,Q560108,NaN,Q560108,P674
2,4962bc3f,What movie won the Oscar for Best Picture in 1...,"[{'name': 'Q102427', 'entityType': 'entity', '...",movies,generic,entity,Q202211,simple,Q102427,1.0,...,"0.794066309928894, 0.20608051121234894, 0.1717...","{'Q102427': 'P8115, P2910, P910, P6262, P1417,...",Q102427,winner,P1346,http://www.wikidata.org/entity/Q61448040,Q61448040,NaN,Q61448040,"R166, R1411"
3,13eaddb4,Who won Best Actress at the Oscars in 2005?,"[{'name': 'Q19020', 'entityType': 'entity', 'l...",movies,generic,entity,Q93187,simple,"Q19020, Q103618",2.0,...,"0.45011794567108154, 0.2730962038040161, 0.181...","{'Q103618': 'P2910, P910, P2517, P6262, P1417,...",Q103618,winner,P1346,"http://www.wikidata.org/entity/Q873, http://ww...","Q873, Q3925, Q4636, Q8927, Q12003, Q14441, Q18...",NaN,"Q873, Q3925, Q4636, Q8927, Q12003, Q14441, Q18...",P1346
4,32cd4730,Who is the actor that played John Kramer in Saw?,"[{'name': 'Q12320195', 'entityType': 'entity',...",movies,generic,entity,Q310190,simple,"Q12320195, Q1145690",2.0,...,"0.8891286849975586, 0.30023348331451416, 0.126...","{'Q486239': 'P11460, P462, P1237, P8796, P1562...",Q486239,screenwriter,P58,"http://www.wikidata.org/entity/Q471215, http:/...","Q471215, Q108047434","http://www.wikidata.org/entity/Q192165, http:/...","Q192165, Q231438, Q233313, Q234344, Q265547, Q...",P161


In [54]:
import requests

def run_query_object(entity_idx, property_idx):
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT ?object  WHERE {

   wd:<ENTITY> wdt:<PROPERTY> ?object .} 
        """.replace(
            "<ENTITY>", entity_idx).replace(
            "<PROPERTY>", property_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    objects = []
    for q in range(len(data["results"]['bindings'])):
        objects.append(data["results"]['bindings'][q]['object']['value'])                 
    return ', '.join(objects)

In [97]:
def run_query_subject(entity_idx, property_idx):
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT ?object  WHERE {

   ?object  wdt:<PROPERTY> wd:<ENTITY> .} 
        """.replace(
            "<ENTITY>", entity_idx).replace(
            "<PROPERTY>", property_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    objects = []
    for q in range(len(data["results"]['bindings'])):
        objects.append(data["results"]['bindings'][q]['object']['value'])                 
    return ', '.join(objects)

In [87]:
data['cand_prop'] = np.nan

In [107]:
for i in tqdm(range(len(data))):
    if pd.notna(data.loc[i, 'true_props']) and pd.isna(data.loc[i, 'cand_prop']): 
    #and len(data.loc[i, 'ents_lbls'].split(', ')) > 1:
        subj = data.loc[i, 'ents_lbls'].split(', ')[1]
        props = data.loc[i, 'true_props'].split(', ')
        for prop in props:
#             prop = data.loc[i, 'true_props'].split(', ')[0]
            time.sleep(1)
            prop = 'P'+prop[1:]
            ans = run_query_subject(subj, prop)
            if len(ans.split(', ')) == 1 and 'entity' in ans:
                data.loc[i, 'cand_prop'] = prop
            else:
                data.loc[i, 'cand_prop'] = np.nan

100%|██████████| 340/340 [02:29<00:00,  2.28it/s]


In [136]:
data.to_csv('mintaka_with_true_props_cands.csv', index = False)